In [6]:
import numpy as np
import requests

# Function to fetch carbon intensities for a specific zone on the previous day
def fetch_carbon_intensities(zone):
    url = "https://api.electricitymap.org/v3/carbon-intensity/history"
    params = {
        "zone": zone
    }
    headers = {
        "auth-token": "6amfE0YN0EGbx"
    }
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        data = response.json()
        carbon_intensities = [entry["carbonIntensity"] for entry in data["history"]]
        return carbon_intensities[-24:]  # Return only the last 24 hours
    else:
        print("Failed to fetch carbon intensities.")
        return None

# Carbon intensities for each zone on the previous day
carbon_intensities_de = fetch_carbon_intensities("DE")
carbon_intensities_it = fetch_carbon_intensities("IT")
carbon_intensities_pt = fetch_carbon_intensities("PT")

# If any of the values is None, it means the fetch failed
if None in (carbon_intensities_de, carbon_intensities_it, carbon_intensities_pt):
    print("Failed to fetch carbon intensities for one or more zones.")
    exit()

# Carbon emissions values for each zone
print("Carbon intensities for Germany:", carbon_intensities_de)
print("Carbon intensities for Italy:", carbon_intensities_it)
print("Carbon intensities for Portugal:", carbon_intensities_pt)
print()

# Example charging data for three companies (24 hours)
charging_company_1 = np.array([5, 6, 5, 6, 7, 8, 9, 10, 9, 8, 7, 6, 5, 6, 7, 8, 9, 10, 9, 8, 7, 6, 5, 4])
charging_company_2 = np.array([4, 5, 4, 5, 6, 7, 8, 9, 8, 7, 6, 5, 4, 5, 6, 7, 8, 9, 8, 7, 6, 5, 4, 3])
charging_company_3 = np.array([3, 4, 3, 4, 5, 6, 7, 8, 7, 6, 5, 4, 3, 4, 5, 6, 7, 8, 7, 6, 5, 4, 3, 2])

# Function to calculate total daily emissions
def calculate_daily_emissions(charging, emissions):
    hourly_emissions = charging * emissions
    daily_emissions = np.sum(hourly_emissions)
    return daily_emissions

# Function to calculate emission scenarios
def calculate_scenarios(total_charging, emissions):
    emissions_sorted_asc = np.sort(emissions)
    emissions_sorted_desc = np.sort(emissions)[::-1]
    
    hourly_capacity = 10  # Define hourly capacity as 10
    hours_needed = int(np.ceil(total_charging / hourly_capacity))
    
    best_case = hourly_capacity * emissions_sorted_asc[:hours_needed]
    worst_case = hourly_capacity * emissions_sorted_desc[:hours_needed]
    
    best_case_total = np.sum(best_case)
    worst_case_total = np.sum(worst_case)
    
    return best_case_total, worst_case_total

# Calculate total charging for each company
total_charging_company_1 = np.sum(charging_company_1)
total_charging_company_2 = np.sum(charging_company_2)
total_charging_company_3 = np.sum(charging_company_3)

# Calculate actual emissions, best case and worst case for each company
emissions_company_1 = calculate_daily_emissions(charging_company_1, carbon_intensities_de)
emissions_company_2 = calculate_daily_emissions(charging_company_2, carbon_intensities_it)
emissions_company_3 = calculate_daily_emissions(charging_company_3, carbon_intensities_pt)

best_case_1, worst_case_1 = calculate_scenarios(total_charging_company_1, carbon_intensities_de)
best_case_2, worst_case_2 = calculate_scenarios(total_charging_company_2, carbon_intensities_it)
best_case_3, worst_case_3 = calculate_scenarios(total_charging_company_3, carbon_intensities_pt)

# Calculate the score using the new approach
def calculate_score(actual_emissions, best_case, worst_case):
    return (actual_emissions - best_case) / (worst_case - best_case)

score_1 = calculate_score(emissions_company_1, best_case_1, worst_case_1)
score_2 = calculate_score(emissions_company_2, best_case_2, worst_case_2)
score_3 = calculate_score(emissions_company_3, best_case_3, worst_case_3)

# Rank the companies based on the new score
companies = ['Company 1', 'Company 2', 'Company 3']
scores = [score_1, score_2, score_3]
ranking = sorted(zip(companies, scores), key=lambda x: x[1])

print("\nOverall company ranking:")
for i, (company, score) in enumerate(ranking):
    print(f"{i + 1}. {company}: Score {score:.2f}")

# Calculate detailed percentages
def calculate_percentages(actual_emissions, best_case, worst_case):
    above_best = ((actual_emissions - best_case) / best_case) * 100
    below_worst = ((worst_case - actual_emissions) / worst_case) * 100
    return above_best, below_worst

percent_above_best_1, percent_below_worst_1 = calculate_percentages(emissions_company_1, best_case_1, worst_case_1)
percent_above_best_2, percent_below_worst_2 = calculate_percentages(emissions_company_2, best_case_2, worst_case_2)
percent_above_best_3, percent_below_worst_3 = calculate_percentages(emissions_company_3, best_case_3, worst_case_3)

# Detailed results
print("\nDetailed scores:")
for company, score, above_best, below_worst in zip(
        companies, scores,
        [percent_above_best_1, percent_above_best_2, percent_above_best_3],
        [percent_below_worst_1, percent_below_worst_2, percent_below_worst_3]):
    print(f"{company}: Score {score:.2f}, {above_best:.2f}% above the best scenario, {below_worst:.2f}% below the worst scenario")


Carbon intensities for Germany: [331, 407, 459, 486, 488, 462, 421, 397, 385, 382, 391, 426, 447, 421, 364, 292, 243, 214, 209, 213, 215, 233, 274, 335]
Carbon intensities for Italy: [217, 247, 264, 267, 252, 238, 215, 201, 202, 204, 208, 218, 232, 237, 219, 197, 185, 170, 160, 158, 166, 167, 203, 231]
Carbon intensities for Portugal: [66, 77, 93, 90, 90, 94, 84, 77, 79, 79, 81, 83, 74, 72, 75, 88, 81, 74, 73, 74, 74, 73, 74, 71]


Overall company ranking:
1. Company 3: Score 0.10
2. Company 2: Score 0.24
3. Company 1: Score 0.38

Detailed scores:
Company 1: Score 0.38, 11.50% above the best scenario, 14.18% below the worst scenario
Company 2: Score 0.24, 4.91% above the best scenario, 12.74% below the worst scenario
Company 3: Score 0.10, 1.47% above the best scenario, 11.68% below the worst scenario
